In [22]:
%%writefile face.py
import cv2
import mediapipe as mp

def main():
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    mp_hands = mp.solutions.hands

    # For static images:
    IMAGE_FILES = []
    with mp_hands.Hands(
        static_image_mode=True,
        max_num_hands=2,
        min_detection_confidence=0.5) as hands:
      for idx, file in enumerate(IMAGE_FILES):
        # Read an image, flip it around y-axis for correct handedness output (see
        # above).
        image = cv2.flip(cv2.imread(file), 1)
        # Convert the BGR image to RGB before processing.
        results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        # Print handedness and draw hand landmarks on the image.
        print('Handedness:', results.multi_handedness)
        if not results.multi_hand_landmarks:
          continue
        image_height, image_width, _ = image.shape
        annotated_image = image.copy()
        for hand_landmarks in results.multi_hand_landmarks:
          print('hand_landmarks:', hand_landmarks)
          print(
              f'Index finger tip coordinates: (',
              f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
              f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height})'
          )
          mp_drawing.draw_landmarks(
              annotated_image,
              hand_landmarks,
              mp_hands.HAND_CONNECTIONS,
              mp_drawing_styles.get_default_hand_landmarks_style(),
              mp_drawing_styles.get_default_hand_connections_style())
        cv2.imwrite(
            '/tmp/annotated_image' + str(idx) + '.png', cv2.flip(annotated_image, 1))
        # Draw hand world landmarks.
        if not results.multi_hand_world_landmarks:
          continue
        for hand_world_landmarks in results.multi_hand_world_landmarks:
          mp_drawing.plot_landmarks(
            hand_world_landmarks, mp_hands.HAND_CONNECTIONS, azimuth=5)

    # For webcam input:
    cap = cv2.VideoCapture(0)
    with mp_hands.Hands(
        model_complexity=0,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:
      while cap.isOpened():
        success, image = cap.read()
        if not success:
          print("Ignoring empty camera frame.")
          # If loading a video, use 'break' instead of 'continue'.
          continue

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)

        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
          for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                image,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
        if cv2.waitKey(5) & 0xFF == 27:
          break
    cap.release()

if __name__ == "__main__":
    main()

Writing face.py


In [23]:
%%writefile cli.py
from face import main

if __name__ == '__main__':
    main()

Writing cli.py


In [24]:
# !pip install opencv-python
# !pip install mediapipe
# !pip install pyinstaller

In [16]:
!python cli.py

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [25]:
!pyinstaller cli.py --add-data="C:\Anaconda\Lib\site-packages\mediapipe\modules;mediapipe/modules" -F -w

1173 INFO: PyInstaller: 4.10
1173 INFO: Python: 3.8.5 (conda)
1173 INFO: Platform: Windows-10-10.0.19041-SP0
1173 INFO: wrote C:\Users\kriku\cli.spec
1176 INFO: UPX is not available.
1180 INFO: Extending PYTHONPATH with paths
['C:\\Users\\kriku']
1618 INFO: checking Analysis
1618 INFO: Building Analysis because Analysis-00.toc is non existent
1618 INFO: Initializing module dependency graph...
1621 INFO: Caching module graph hooks...
1633 INFO: Analyzing base_library.zip ...
5384 INFO: Processing pre-find module path hook distutils from 'c:\\anaconda\\lib\\site-packages\\PyInstaller\\hooks\\pre_find_module_path\\hook-distutils.py'.
5384 INFO: distutils: retargeting to non-venv dir 'c:\\anaconda\\lib'
8176 INFO: Caching module dependency graph...
8342 INFO: running Analysis Analysis-00.toc
8355 INFO: Adding Microsoft.Windows.Common-Controls to dependent assemblies of final executable
  required by c:\anaconda\python.exe
8532 INFO: Analyzing C:\Users\kriku\cli.py
10837 INFO: Processing pr